The aim of this notebook is to explore the cite-seq data and point to specific parts of the data to be used for evaluation of protein expression prediction from RNA.

The object that we're working with is an MuData object ([https://anndata.readthedocs.io/en/latest/tutorials/notebooks/getting-started.html ](https://mudata.readthedocs.io/en/latest/notebooks/quickstart_mudata.html)- for more details). 

Both RNA and protein expression are contained in MuData as seperate AnnData objects.

We'll start by loading the necessary libraries (should have been installed with conda env install) and then looking at the data.

In [1]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import scanpy as sc
import mudata as md 
from mudata import MuData
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
import sys
sys.path.append('..')
from utils import load_appris

In [2]:
# Load the MuData object (This is a big file - should have at last 16GB of RAM to prevent memory issues)
h5_path = '/fs01/projects/isoclr/multi.h5mu'
mdata = md.read(h5_path)
surface_proteins = mdata["ADT"].var.index.values
surface_proteins

/h/phil/anaconda3/envs/cite_seq/lib/python3.12/site-packages/mudata/_core/mudata.py:489: UserWarning: Cannot join columns with the same name because var_names are intersecting.
  warnings.warn(


array(['CD39', 'Rat-IgG1-1', 'CD107a', 'CD62P', 'TCR-2', 'CD30', 'CD31',
       'CD34', 'CD35', 'CD36', 'CD223', 'TIGIT', 'TCR-V-9', 'CD226',
       'CD178', 'CD319', 'CD171', 'Siglec-8', 'CD340', 'Rat-IgG2b',
       'VEGFR-3', 'CD29', 'CD62E', 'CD4-2', 'CD4-1', 'CD22', 'CD3-1',
       'CD20', 'CD27', 'CD45RB', 'CD25', 'CD24', 'CD146', 'Galectin-9',
       'CD142', 'CD141', 'CD294', 'Rat-IgG1-2', 'CD45RA', 'CX3CR1',
       'CD56-2', 'CD56-1', 'CD45RO', 'CD303', 'GP130', 'CD253', 'CD357',
       'CD11b-1', 'CD354', 'CD11b-2', 'CLEC12A', 'CD38-2', 'CD38-1',
       'Folate', 'Rag-IgG2c', 'CD209', 'CD152', 'CD154', 'CD155',
       'Cadherin', 'CD201', 'CD204', 'CD205', 'CD206', 'CD207', 'CD1d',
       'CD284', 'CD1c', 'Podoplanin', 'CD1a', 'CD366', 'IgD', 'IgM',
       'CD66a/c/e', 'CD49d', 'LOX-1', 'TIM-4', 'CD98', 'CD370', 'CD49a',
       'CD44-2', 'C5L2', 'CD44-1', 'CD158e1', 'CD124', 'CD127', 'CD126',
       'CD279', 'CD278', 'CD123', 'CD122', 'CD96', 'CD274', 'CD95',
       'CD271', '

From here, we can see that the data is separated as 'two modalities', which are SCT (RNA) and ADT (Surface protein expression)

There are further subsets of data for `highly_variable`, which are essentially genes that vary across the cells. 

## Load Gencode

In [3]:
gencode_df = pd.read_csv(
    '/scratch/hdd001/home/phil/rna_contrast/datasets/data_for_ian/'
    'annotation_data/refseq_gencode_files/human_comprehensive_gencode_v41_hg38.tsv', 
    sep='\t'
)
set_of_genes = set(gencode_df.name2)
gencode_intersect = set_of_genes.intersection(set(surface_proteins))
len(gencode_intersect)

45

## Perform CD - gene symbol mapping

In [4]:

import requests
manual_map = {
    'Siglec-8': 'SIGLEC8',
    'Galectin-9': 'LGALS9',
    'TIM-4': 'TIM',
    'Notch-1': 'NOTCH1',
    'Notch-2': 'NOTCH2',
    'Integrin-7': 'ITGA7',
}
blacklist = ['Rat-IgG1', 'TCR']

def get_ensembl_gene_info(cd_name, species='human'):
    for entry in blacklist:
        if entry in cd_name:
            return None, None
        
    for key, value in manual_map.items():
        if key in cd_name:
            cd_name = value
            break
    
    if '-' in cd_name and 'CD' in cd_name:
        cd_name = cd_name.split('-')[0]

    server = 'https://rest.ensembl.org'
    ext = f'/xrefs/symbol/{species}/{cd_name}?'

    headers = {'Content-Type': 'application/json'}

    response = requests.get(server + ext, headers=headers)

    if not response.ok:
        return None, None

    data = response.json()

    for entry in data:
        if entry['type'] == 'gene':
            if 'id' not in entry:
                ensembl_id = None
            else:
                ensembl_id = entry['id']
                
            if 'display_id' not in entry:
                gene_symbol = None
            else:
                gene_symbol = entry['display_id']
                
            return ensembl_id, gene_symbol

    return None, None

import requests

def get_gene_symbol_from_ensembl_id(ensembl_id):
    """
    Retrieves the gene symbol for a given Ensembl gene ID using the Ensembl REST API.

    Parameters:
        ensembl_id (str): The Ensembl gene ID (e.g., 'ENSG00000115884').

    Returns:
        str or None: The gene symbol if found; otherwise, None.
    """
    server = 'https://rest.ensembl.org'
    ext = f'/lookup/id/{ensembl_id}?content-type=application/json'
    
    response = requests.get(server + ext)
    
    if not response.ok:
        # Handle HTTP errors
        print(f"Error: Unable to retrieve data for Ensembl ID {ensembl_id}")
        return None
    
    try:
        data = response.json()
    except ValueError:
        # Handle JSON decoding errors
        print("Error: Received invalid JSON response")
        return None
    
    gene_symbol = data.get('display_name')
    
    if gene_symbol:
        return gene_symbol
    else:
        print(f"Gene symbol not found for Ensembl ID {ensembl_id}")
        return None



# Example usage
cd_name = 'TIM'
ensembl_id, gene_symbol = get_ensembl_gene_info(cd_name)

print(f'CD Name: {cd_name}')
print(f'Ensembl ID: {ensembl_id}')
print(f'Gene Symbol: {gene_symbol}')

# Example usage
gene_symbol = get_gene_symbol_from_ensembl_id(ensembl_id)

print(f'Ensembl ID: {ensembl_id}')
print(f'Gene Symbol: {gene_symbol}')



CD Name: TIM
Ensembl ID: ENSG00000050327
Gene Symbol: None
Ensembl ID: ENSG00000050327
Gene Symbol: ARHGEF5


In [5]:
data = []
for gene in tqdm(surface_proteins):
    ensembl_id, gene_symbol = get_ensembl_gene_info(gene)
    if not ensembl_id:
        continue
    gene_symbol = get_gene_symbol_from_ensembl_id(ensembl_id)
    data.append({
        'gene': gene,
        'ensembl_id': ensembl_id,
        'gene_symbol': gene_symbol,
    })
    
gene_df = pd.DataFrame(data)
len(gene_df['gene_symbol'].unique())
appris = load_appris()
gene_df = gene_df.merge(appris[['Gene name', 'Transcript ID']], left_on='gene_symbol', right_on='Gene name', how='inner')
gene_df['transcript_id'] = gene_df['Transcript ID']
gene_df.drop(columns=['Gene name', 'Transcript ID'], inplace=True)


  8%|▊         | 18/228 [00:12<02:42,  1.29it/s]

 81%|████████  | 185/228 [02:43<00:40,  1.05it/s]

Gene symbol not found for Ensembl ID ENSG00000284969


100%|██████████| 228/228 [03:19<00:00,  1.14it/s]


15791


In [8]:
list_of_genes = list(set(gencode_df['name2']).intersection(set(gene_df['gene_symbol'])))

orthrus_path = '/fs01/projects/isoclr/comprehensive_gencode_isoclr_large_orth_splice/'
files_in_orthrus_path = os.listdir(orthrus_path)

sub_dfs = []
for f in tqdm(files_in_orthrus_path):
    df = pd.read_csv(orthrus_path + f, compression='gzip')
    df['transcript_id'] = df['transcript_name'].str.split('.').str[0]
    sub_dfs.append(df[df['transcript_id'].isin(gene_df['transcript_id'])])

orthrus_df = pd.concat(sub_dfs, axis=0)
orthrus_df = orthrus_df.merge(gene_df, on='transcript_id', how='inner')
orthrus_df = orthrus_df.drop(columns=['Unnamed: 0'])
orthrus_df = orthrus_df.rename(columns={f"{i}": f'orthrus_{i}' for i in range(512)})

100%|██████████| 218/218 [00:35<00:00,  6.10it/s]


In [9]:
mdata['ADT'].var

,highly_variable
CD39,1
Rat-IgG1-1,1
CD107a,1
CD62P,1
TCR-2,1
...,...
CD164,1
CD138-2,1
CD144,1
CD202b,1


In [17]:
orthrus_df

,orthrus_0,orthrus_1,orthrus_2,orthrus_3,orthrus_4,orthrus_5,orthrus_6,orthrus_7,orthrus_8,orthrus_9,...,gene_name,GC_percentage,utr5_len,cds_len,utr3_len,n_exons,transcript_id,gene,ensembl_id,gene_symbol
0,0.251251,0.139169,0.010113,-0.149337,-0.037124,-0.179202,-0.080573,-0.138957,-0.486310,0.475147,...,CD96,0.312153,52,1710,2566,14,ENST00000352690,CD96,ENSG00000153283,CD96
1,0.113176,0.087325,0.066683,-0.261741,-0.142903,-0.286960,-0.133052,-0.145283,-0.516602,0.436270,...,TNFRSF18,0.649123,21,726,336,5,ENST00000379268,CD357,ENSG00000186891,TNFRSF18
2,0.105992,0.156068,0.052590,-0.321472,-0.192780,-0.235153,-0.124217,-0.089109,-0.542223,0.453860,...,TNFRSF4,0.693023,26,834,215,7,ENST00000379236,CD134,ENSG00000186827,TNFRSF4
3,0.109393,0.154513,0.033611,-0.233739,-0.087119,-0.220370,-0.120727,-0.106481,-0.541486,0.411814,...,TNFRSF14,0.601058,294,852,556,8,ENST00000355716,CD270,ENSG00000157873,TNFRSF14
4,0.160029,0.132224,0.025293,-0.225839,-0.133532,-0.206766,-0.111233,-0.087655,-0.495147,0.421889,...,TNFRSF9,0.199080,140,768,4964,8,ENST00000377507,CD137,ENSG00000049249,TNFRSF9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
185,0.173817,0.095276,0.081000,-0.223594,-0.017731,-0.116691,0.089400,-0.193010,-0.239946,0.406139,...,FUT4,0.389623,213,1593,4169,1,ENST00000358752,CD15,ENSG00000196371,FUT4
186,0.096913,0.245783,-0.068318,-0.144862,-0.058341,-0.182594,-0.067333,-0.190562,-0.496894,0.319785,...,LILRA4,0.569018,69,1500,387,8,ENST00000291759,CD85g,ENSG00000239961,LILRA4
187,0.133407,0.130347,-0.063831,-0.059053,0.083969,-0.295212,-0.149313,-0.212894,-0.523098,0.217535,...,LAIR1,0.531016,61,813,593,9,ENST00000348231,CD305,ENSG00000167613,LAIR1
188,0.126054,0.168476,-0.002600,0.036611,0.084658,-0.094854,-0.056073,-0.143101,-0.449250,0.225876,...,KIR3DL1,0.464992,33,1335,503,9,ENST00000391728,CD158e1,ENSG00000167633,KIR3DL1


In [19]:
pd.read_csv('../data/esm_data.csv')

,gene,transcript,esm_0,esm_1,esm_2,esm_3,esm_4,esm_5,esm_6,esm_7,...,esm_310,esm_311,esm_312,esm_313,esm_314,esm_315,esm_316,esm_317,esm_318,esm_319
0,PTGDR2,ENST00000332539.4,0.194653,0.126780,0.182600,-0.029803,0.051965,0.026792,0.109676,-0.213097,...,0.128914,0.092471,0.115099,0.152762,0.111469,0.124649,0.054852,0.122809,-0.076046,-0.050964
1,MERTK,ENST00000295408.8,0.174655,0.162040,0.159292,-0.064161,0.037378,0.025869,0.214430,-0.168868,...,0.130357,0.049052,0.103070,0.135618,0.163095,-0.009734,0.029290,0.162163,-0.071493,0.131510
2,CD46,ENST00000358170.6,0.160673,0.180229,0.124241,-0.047515,0.121338,0.048503,0.204651,-0.230932,...,0.178895,0.039712,0.099157,-0.035398,0.227750,-0.011657,0.019348,0.188651,-0.027152,0.146914
3,ITGB3,ENST00000559488.5,0.033419,0.148301,0.171840,-0.122247,0.052923,0.050449,0.140216,-0.211933,...,0.101055,0.028358,0.171624,0.008692,0.149858,-0.025355,0.080045,0.208778,-0.035152,0.119216
4,TNFSF4,ENST00000281834.3,0.159453,0.045351,0.133093,-0.065311,-0.001200,0.090423,0.113904,-0.243310,...,0.225913,0.040134,0.083178,-0.037448,0.193334,-0.017796,0.067195,0.200707,0.061961,0.077091
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
181,THY1,ENST00000284240.9,0.108301,0.146954,0.171239,-0.001321,0.000676,-0.037172,0.190913,-0.116044,...,0.157813,0.002127,0.212422,0.027532,0.151119,0.039846,0.055663,0.189299,-0.071068,0.024650
182,SLC7A5,ENST00000261622.4,0.110616,0.120736,0.166708,-0.035123,0.000360,0.071963,0.124000,-0.362562,...,0.121050,0.019785,0.111488,0.042031,0.230152,-0.065543,0.002094,0.153272,-0.042976,0.102418
183,CD69,ENST00000228434.7,0.197017,0.137450,0.162856,0.019329,0.073427,0.057510,0.155821,-0.286048,...,0.216885,0.054472,0.062408,-0.029178,0.193458,0.010828,0.010842,0.188487,-0.011039,0.043911
184,CXCR6,ENST00000304552.4,0.182692,0.106208,0.124159,-0.011814,0.007098,0.064117,0.131658,-0.237451,...,0.229905,0.046099,0.081270,0.091075,0.220422,-0.002151,0.039243,0.161981,-0.020390,0.000847


In [31]:
new_var = orthrus_df.set_index('gene')
new_var = mdata['ADT'].var.merge(new_var, left_index=True, right_index=True, how='left')
new_var
esm_df = pd.read_csv('../data/esm_data.csv')
new_var = new_var.reset_index()
new_var = new_var.merge(esm_df, left_on='gene_name', right_on='gene', how='left')
new_var = new_var.set_index('index')

In [32]:
assert (new_var.index == mdata['ADT'].var.index).all()
# Inspect the object 
for column in new_var.columns:
    mdata['ADT'].var[column] = new_var[column].copy()

/tmp/ipykernel_19718/2919326676.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  mdata['ADT'].var[column] = new_var[column].copy()
/tmp/ipykernel_19718/2919326676.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  mdata['ADT'].var[column] = new_var[column].copy()
/tmp/ipykernel_19718/2919326676.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de

In [33]:
write_dir = '/fs01/projects/isoclr/'
mdata.write(f'{write_dir}/cite_seq_with_seq_embed.h5mu')

/h/phil/anaconda3/envs/cite_seq/lib/python3.12/site-packages/mudata/_core/mudata.py:489: UserWarning: Cannot join columns with the same name because var_names are intersecting.
  warnings.warn(


In [22]:
# save mdata_object


/h/phil/anaconda3/envs/cite_seq/lib/python3.12/site-packages/mudata/_core/mudata.py:489: UserWarning: Cannot join columns with the same name because var_names are intersecting.
  warnings.warn(


In [13]:
mdata2 = md.read(f'{write_dir}/cite_seq_with_seq_embed.h5mu')

/h/cait/miniconda3/envs/cite_seq/lib/python3.12/site-packages/mudata/_core/mudata.py:489: UserWarning: Cannot join columns with the same name because var_names are intersecting.
  warnings.warn(


In [5]:
mdata.obs # This contains all the metadata, e.g. cell-type label

,nCount_ADT,nFeature_ADT,nCount_RNA,nFeature_RNA,orig.ident,lane,donor,time,celltype.l1,celltype.l2,celltype.l3,Phase,nCount_SCT,nFeature_SCT
L1_AAACCCAAGAAACTCA,7430.0,221,10823.0,2915,P2_7,L1,P2,7,Mono,CD14 Mono,CD14 Mono,G1,6380.0,2548
L1_AAACCCAAGACATACA,5949.0,211,5864.0,1617,P1_7,L1,P1,7,CD4 T,CD4 TCM,CD4 TCM_1,G1,5693.0,1615
L1_AAACCCACAACTGGTT,6547.0,217,5067.0,1381,P4_3,L1,P4,3,CD8 T,CD8 Naive,CD8 Naive,S,5066.0,1379
L1_AAACCCACACGTACTA,3508.0,207,4786.0,1890,P3_7,L1,P3,7,NK,NK,NK_2,G1,4984.0,1889
L1_AAACCCACAGCATACT,6318.0,219,6505.0,1621,P4_7,L1,P4,7,CD8 T,CD8 Naive,CD8 Naive,G1,5854.0,1620
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E2L8_TTTGTTGGTCGTGATT,4089.0,200,9346.0,2201,P5_7,E2L8,P5,7,CD8 T,CD8 Naive,CD8 Naive,S,8231.0,2200
E2L8_TTTGTTGGTGTGCCTG,6869.0,213,9318.0,2938,P5_3,E2L8,P5,3,Mono,CD14 Mono,CD14 Mono,G1,8680.0,2938
E2L8_TTTGTTGGTTAGTTCG,4173.0,208,11619.0,3224,P8_0,E2L8,P8,0,B,B intermediate,B intermediate kappa,S,8555.0,3130
E2L8_TTTGTTGGTTGGCTAT,5979.0,221,15436.0,3999,P5_3,E2L8,P5,3,Mono,CD16 Mono,CD16 Mono,G1,8970.0,3533


We can see the n_vars for ADT is much less (228 proteins) compared to SCT (20,000 genes). We can do a simple analysis on the RNA component of the data and visualize by UMAP and cell-type label

In [13]:
rna_copy = mdata['SCT'].copy()
rna_copy.obs = mdata.obs

In [14]:
sc.pp.normalize_total(rna_copy, target_sum=1e4)
sc.pp.log1p(rna_copy)
sc.pp.highly_variable_genes(rna_copy, n_top_genes=2000)
sc.pp.pca(rna_copy, n_comps=20)
sc.pp.neighbors(rna_copy)
sc.tl.umap(rna_copy)


: 

# Add scgpt cell embeddings

In [21]:
scgpt_adata = md.read('/fs01/projects/isoclr/scgpt_sct.h5ad')

In [46]:
scgpt_df = pd.DataFrame(scgpt_adata.X)
scgpt_df.columns = [f'scgpt_{i+1}' for i in range(len(scgpt_df.columns))]
scgpt_df.index = mdata2['SCT'].obs.index

In [49]:
mdata2['SCT'].obs = pd.concat([mdata2['SCT'].obs, scgpt_df], axis=1)

In [50]:
mdata2['SCT'].obs

,scgpt_1,scgpt_2,scgpt_3,scgpt_4,scgpt_5,scgpt_6,scgpt_7,scgpt_8,scgpt_9,scgpt_10,...,scgpt_503,scgpt_504,scgpt_505,scgpt_506,scgpt_507,scgpt_508,scgpt_509,scgpt_510,scgpt_511,scgpt_512
L1_AAACCCAAGAAACTCA,-0.012128,-0.018793,-0.036334,-0.048442,0.015501,-0.027967,-0.006040,-0.010017,-0.008237,-0.021338,...,0.011750,-0.003717,0.043064,0.007716,-0.025755,-0.041495,0.028571,0.008149,-0.014695,0.037343
L1_AAACCCAAGACATACA,-0.005710,0.015951,-0.034136,-0.053240,-0.011317,-0.018448,-0.002965,0.002213,0.003070,-0.004500,...,-0.018986,-0.022464,0.047461,-0.021659,-0.031254,-0.037813,0.011875,-0.001523,-0.000293,-0.002988
L1_AAACCCACAACTGGTT,-0.001284,0.014184,-0.028089,-0.050108,-0.010221,-0.015381,0.006383,0.003099,0.004475,-0.006451,...,-0.026332,-0.019764,0.038446,-0.016203,-0.032821,-0.035196,0.000661,0.005039,-0.002330,-0.000265
L1_AAACCCACACGTACTA,-0.009032,0.007458,-0.020631,-0.063720,0.034423,-0.019588,-0.001143,0.002864,-0.005935,-0.017691,...,-0.023930,0.019967,0.051261,-0.006669,-0.014930,-0.045922,0.009328,0.022987,0.006883,0.007586
L1_AAACCCACAGCATACT,-0.002890,0.014381,-0.015587,-0.049046,-0.009383,-0.020891,0.008933,-0.001697,0.005184,-0.011500,...,-0.028857,-0.023227,0.034676,-0.008157,-0.023695,-0.031830,-0.006801,0.005746,0.007819,0.006381
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E2L8_TTTGTTGGTCGTGATT,-0.017192,0.010307,-0.035389,-0.051229,-0.000323,-0.027517,0.010682,-0.001544,0.007706,-0.001746,...,-0.030904,-0.014757,0.031411,-0.016673,-0.017157,-0.039095,-0.000485,0.003020,0.000182,-0.003036
E2L8_TTTGTTGGTGTGCCTG,-0.017028,0.009356,-0.063301,-0.038363,-0.018663,-0.030428,0.003105,-0.029220,-0.015046,-0.028040,...,0.017455,-0.004128,0.034745,0.010159,-0.006635,-0.030744,0.053570,0.010452,-0.028271,0.028050
E2L8_TTTGTTGGTTAGTTCG,-0.010546,0.023073,-0.033995,-0.021307,0.038379,-0.030083,-0.004648,0.001651,0.001386,-0.005282,...,-0.035041,-0.024569,0.034850,0.002757,-0.004565,-0.022074,-0.018735,-0.028806,0.000841,-0.000927
E2L8_TTTGTTGGTTGGCTAT,-0.021310,-0.009677,-0.034738,-0.072634,-0.017247,-0.056189,-0.004853,-0.034620,-0.019731,-0.009040,...,0.020351,0.003673,0.063843,0.030745,-0.014088,-0.060559,0.041638,0.008359,-0.029096,0.035669


In [51]:
# save mdata_object
mdata2.write(f'{write_dir}/cite_seq_with_seq_embed_with_cell_embed.h5mu')

/h/cait/miniconda3/envs/cite_seq/lib/python3.12/site-packages/mudata/_core/mudata.py:489: UserWarning: Cannot join columns with the same name because var_names are intersecting.
  warnings.warn(
